In [84]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.enums import Resampling
from rasterio.plot import show
from rasterio.crs import CRS
import strgen
import random

# Load Thailand GEDI data

In [2]:
#### Filter Thailand GEDI path goes here!! ####
th_gedi = gpd.read_file('/home/jupyter/unet_canopyheight_estimation/data/processed/GEDI/GEDI_L2_Thailand_GEDI_2021-2022.gpkg')

In [3]:
th_gedi.head()

,Shot_Number,Beam,Tandem-X DEM,Elevation,CanopyElev,CH(rh95),RH_0,RH_10,RH_20,RH_30,...,DegradeFlag,Sensitivity,Algorithm,Num_modes,Noise_std,Max_Amp,toploc_a1,botloc_a1,NoiseRatio,geometry
0,1.071700e+17,BEAM0000,-27.884317,-35.540466,-15.805820,17.41,-3.63,3.07,5.16,8.01,...,0.0,0.976366,10.0,6.0,3.122070,110.732208,294.50,414.50,35.467564,POINT (98.05993 16.46011)
1,1.071700e+17,BEAM0000,-25.001328,-26.497679,-13.166495,11.38,-4.19,-0.33,2.17,4.79,...,0.0,0.972506,2.0,2.0,2.977822,129.138916,294.00,402.00,43.366905,POINT (98.06026 16.46053)
2,1.071700e+17,BEAM0000,-22.599817,-30.346342,-10.798934,17.07,-2.65,4.41,6.10,8.94,...,0.0,0.979748,10.0,4.0,2.999874,97.871094,293.50,402.75,32.625069,POINT (98.06059 16.46094)
3,1.071700e+17,BEAM0000,-18.195734,-25.073175,-6.349607,16.58,-1.98,2.32,6.47,8.38,...,0.0,0.963239,2.0,3.0,3.118909,100.269989,299.25,428.75,32.149059,POINT (98.06419 16.46545)
4,1.071700e+17,BEAM0000,-18.195734,-24.069311,-14.557739,6.70,-5.46,-2.13,-1.19,-0.63,...,0.0,0.972485,2.0,3.0,3.064579,178.740082,305.75,394.00,58.324524,POINT (98.06452 16.46586)


In [4]:
th_gedi

,Shot_Number,Beam,Tandem-X DEM,Elevation,CanopyElev,CH(rh95),RH_0,RH_10,RH_20,RH_30,...,DegradeFlag,Sensitivity,Algorithm,Num_modes,Noise_std,Max_Amp,toploc_a1,botloc_a1,NoiseRatio,geometry
0,1.071700e+17,BEAM0000,-27.884317,-35.540466,-15.805820,17.410000,-3.63,3.070000,5.160000,8.010000,...,0.0,0.976366,10.0,6.0,3.122070,110.732208,294.50,414.50,35.467564,POINT (98.05993 16.46011)
1,1.071700e+17,BEAM0000,-25.001328,-26.497679,-13.166495,11.380000,-4.19,-0.330000,2.170000,4.790000,...,0.0,0.972506,2.0,2.0,2.977822,129.138916,294.00,402.00,43.366905,POINT (98.06026 16.46053)
2,1.071700e+17,BEAM0000,-22.599817,-30.346342,-10.798934,17.070000,-2.65,4.410000,6.100000,8.940000,...,0.0,0.979748,10.0,4.0,2.999874,97.871094,293.50,402.75,32.625069,POINT (98.06059 16.46094)
3,1.071700e+17,BEAM0000,-18.195734,-25.073175,-6.349607,16.580000,-1.98,2.320000,6.470000,8.380000,...,0.0,0.963239,2.0,3.0,3.118909,100.269989,299.25,428.75,32.149059,POINT (98.06419 16.46545)
4,1.071700e+17,BEAM0000,-18.195734,-24.069311,-14.557739,6.700000,-5.46,-2.130000,-1.190000,-0.630000,...,0.0,0.972485,2.0,3.0,3.064579,178.740082,305.75,394.00,58.324524,POINT (98.06452 16.46586)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4322969,1.347711e+17,BEAM1011,237.048721,168.696823,237.287552,62.740002,-0.56,32.470001,39.669998,43.560001,...,0.0,0.994533,5.0,15.0,2.791038,104.707977,293.00,530.25,37.515789,POINT (105.00418 20.30980)
4322970,1.347711e+17,BEAM1011,351.898804,346.264496,363.758728,14.600000,-4.98,0.030000,2.390000,4.300000,...,0.0,0.991713,10.0,4.0,2.862891,93.046875,291.75,428.75,32.501022,POINT (105.00692 20.31304)
4322971,1.347711e+17,BEAM1011,416.093323,414.550446,433.580627,16.629999,-8.80,2.020000,6.030000,8.870000,...,0.0,0.987732,10.0,3.0,2.988071,145.358032,293.50,425.75,48.646114,POINT (105.00863 20.31506)
4322972,1.347711e+17,BEAM1011,333.123016,326.686920,338.562042,6.810000,-11.12,-6.290000,-3.890000,-2.360000,...,0.0,0.963486,1.0,1.0,2.723582,171.813049,284.75,438.25,63.083496,POINT (105.08831 20.40879)


# Process GEDI data
## Flow</br>
for eachtile</br>
>load tile shapefile</br>
>extract tilename
>extract boundary</br>
>create new dataframe --- no?</br>
>filter GEDI within boundary and keep in new dataframe</br>
>Use tile name to open</br>
>>open sentinel 2 SCL</br>
>>open max NDVI map</br>
>>open canopy cover 2000 map</br>
>>open canopy cover 2010 map</br>
>>open ESA map</br>

>Assign Canopy height to RH 95</br>
>Get Coordinate of GEDI</br>
>Get Canopy cover data</br>
>Get ESA Classification data</br>
>Get Maximun anuual NDVI  data</br>
>get NDVI outlier</br>
>get SCL data</br>
>relabel GEDI data (Waveform filtration) if GEDI data fall into case of non forest</br>
>>create flag arttibute to mask non forest GEDI for make validation data later</br>

>save data to gopackage format base on tilename</br>
     
     

In [5]:
# Boundary Shapfile root
extent_shapefile = '/home/jupyter/Sentinel2_Data/Boundary_Shapefile'
# canopy cover root
treecov_dir = '/home/jupyter/Sentinel2_Data/CanopyCover_mask'
# ESA world cover mask root
esa_dir = '/home/jupyter/Sentinel2_Data/ESA_Mask_32647'
# Maximun NDVI root
max_ndvi_root= '/home/jupyter/Sentinel2_Data/Max_NDVI'

# target output dir
target_dir = '/home/jupyter/Sentinel2_Data/Filter_GEDI'

In [ ]:
for root, dirs, files in os.walk(extent_shapefile, topdown=True):
    #print(root)
    for fname in files:
        if fname.endswith('shp_32647.gpkg'):
            print(fname)
            # Load tile shapfile
            tile_shapefile = gpd.read_file(os.path.join(root,fname))
            tile_shapefile = tile_shapefile.to_crs(4326)
            # Extract tilename
            tilename =  fname.split('_')[0] # Ex: 47PMS
            # filter GEDI within boundary and keep in new dataframe
            #minLon, minLat, maxLon, maxLat = tile_shapefile.envelope[0].bounds
            gedi_subset = th_gedi.copy()
            gedi_subset = gpd.sjoin(
                        gedi_subset,
                        tile_shapefile,
                        how='inner',
                        predicate='within')
            #gedi_subset = gedi_subset.dropna()  # Drop shots outside of the ROI
            gedi_subset = gedi_subset.reset_index(drop=True)
            gedi_subset = gedi_subset.to_crs(32647)
            
            
            
            
            # Use tile name to open filter data
            
            # Landsat 2000,2010 tree cover
            treecov_files = os.listdir(treecov_dir)
            #open canopy cover 2000 map
            treecov_2000_match = [img for img in treecov_files if (tilename in img) and ('2000' in img)][0] # extract filename
            canopy_cov_2000_path = os.path.join(treecov_dir,treecov_2000_match)
            canopy_cov_2000 = rasterio.open(canopy_cov_2000_path)
            #open canopy cover 2010 map
            treecov_2010_match = [img for img in treecov_files if (tilename in img) and ('2010' in img)][0] # extract filename
            canopy_cov_2010_path = os.path.join(treecov_dir,treecov_2010_match)
            canopy_cov_2010 = rasterio.open(canopy_cov_2010_path)
            
            # ESA world cover
            esa_files = os.listdir(esa_dir)
            #open ESA world cover map
            esa_match = [img for img in esa_files if (tilename in img)][0] # extract filename
            esa = rasterio.open(os.path.join(esa_dir,esa_match))
            
            # Maximun annual NDVI data
            ndvi_files = os.listdir(max_ndvi_root)
            #open max ndvi img
            ndvi_match = [img for img in ndvi_files if (tilename in img)][0] # extract filename
            ndvi_max = rasterio.open(os.path.join(max_ndvi_root,ndvi_match))
            
            #Assign Canopy height to RH 95
            gedi_subset['Canopy_height'] = gedi_subset['RH_95']
            
            #Get Coordinate of GEDI
            coord_list = [(x,y) for x,y in zip(gedi_subset['geometry'].x , gedi_subset['geometry'].y)]
            
            ### Get Canopy cover data
            gedi_subset['TreeCov2000'] = [x[0] for x in canopy_cov_2000.sample(coord_list)]
            gedi_subset['TreeCov2010'] = [x[0] for x in canopy_cov_2010.sample(coord_list)]
            
            ## Get ESA Classification data
            gedi_subset['ESA_CLS'] = [x[0] for x in esa.sample(coord_list)]
            
            ## Get Maximum anuual NDVI  data
            gedi_subset['NDVI_Max'] = [x[0] for x in ndvi_max.sample(coord_list)]
            
            #Relabel GEDI data (Waveform filtration) if GEDI data fall into case of non forest
            
            # Create flage to track if there point is not canopy cover 0 = tree_cover , 1 = urban_area
            gedi_subset.loc[:,'urban_flag'] = 0
            
            #### Assign GEDI Sample with RH95 < 3 m to Zero
            gedi_subset.loc[(gedi_subset['Canopy_height'] <= 3) ,'Canopy_height'] = 0.0
            
            # if no tree cover in both 2000 , 2010: set value to zero if match one of following citeria:
            # 1.Pixel is classify as water (ESA == 80)
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 80) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 80) , 'urban_flag'] = 1
            
            # 2. Pixel locate in urban area and Maximum annual NDVI below 0.5
            # 50 Built up
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 50) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 50) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 60 Sparse Vegetation
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 60) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 60) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 40 Crop land
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 40) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 40) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 30 Grass land
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 30) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 30) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 20 Shrub land
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 20) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['ESA_CLS'] == 20) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            
            # 3.Maximum annual NDVI below 0.3
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['NDVI_Max'] < 0.3) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] < 20) & (gedi_subset['TreeCov2010'] < 20) & (gedi_subset['NDVI_Max'] < 0.3) , 'urban_flag'] = 1
            
            # Or Tree cover is identify as Water Cloud Shadow (>= 200)
            # 1.Pixel is classify as water (ESA == 80)
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 80) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 80) , 'urban_flag'] = 1

            # 2. Pixel locate in urban area and Maximum annual NDVI below 0.5
            # 50 Built up
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 50) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 50) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 60 Sparse Vegetation
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 60) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 60) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 40 Crop land
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 40) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 40) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 30 Grass land
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 30) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 30) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1
            # 20 Shrub land
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 20) & (gedi_subset['NDVI_Max'] < 0.5) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['ESA_CLS'] == 20) & (gedi_subset['NDVI_Max'] < 0.5) , 'urban_flag'] = 1

            # 3.Maximum annual NDVI below 0.3
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['NDVI_Max'] < 0.3) , 'Canopy_height'] = 0.0
            gedi_subset.loc[(gedi_subset['TreeCov2000'] >= 200) & (gedi_subset['TreeCov2010'] >= 200) & (gedi_subset['NDVI_Max'] < 0.3) , 'urban_flag'] = 1
            
            # drop any Nan value
            gedi_subset = gedi_subset.dropna()
            gedi_subset = gedi_subset.reset_index(drop=True)
            
            ## save file
            outName = tilename + '_GEDI.gpkg'
            gedi_subset.to_file(os.path.join(target_dir,outName), driver="GPKG")
            

47PMS_composite_norm_boundary_shp_32647.gpkg
47PRR_composite_norm_boundary_shp_32647.gpkg
47PNS_composite_norm_boundary_shp_32647.gpkg
47PMT_composite_norm_boundary_shp_32647.gpkg
47QMB_composite_norm_boundary_shp_32647.gpkg
47PQQ_composite_norm_boundary_shp_32647.gpkg
47PQS_composite_norm_boundary_shp_32647.gpkg
47QMA_composite_norm_boundary_shp_32647.gpkg


# Rasterize GEDI data

## Create empyty raster
## Flow</br>
for eachtile</br>
>load median composite norm GeoTiFF</br>
>extract tilename</br>
>extract geo transformation</br>
>extract crs</br>
>create empty array and fill with 0</br>
>write empty raster and and name with tilename</br>

In [ ]:
reference_rootdir = '/home/jupyter/Sentinel2_Data/Composite_Normalize'
target_empty_raster = '/home/jupyter/Sentinel2_Data/GEDI_empty_raster'

In [ ]:
for root, dirs, files in os.walk(reference_rootdir, topdown=True):
    #print(root)
    for fname in files:
        if fname.endswith('.tif'):
            print(fname)
            ref_img = rasterio.open(os.path.join(root,fname))
            # Extract tilename
            tilename =  fname.split('_')[0]
            # Extract geo transformation
            geo_transform = ref_img.transform
            # Extract crs
            crs = ref_img.crs
            
            # Create empty raster
            empty_arr = np.full((10980, 10980), 0)
            new_dataset = rasterio.open(os.path.join(target_empty_raster,f'{tilename}_empty.tif'), 'w', driver='GTiff',
                            height = empty_arr.shape[0], width = empty_arr.shape[1],
                            count=1, dtype='float64',
                            crs=crs,
                            transform=geo_transform)
            # write empty raster
            new_dataset.write(empty_arr, 1)
            new_dataset.close()

## Plot GEDI footprint to raster file
## Flow</br>
for eachtile</br>
>load empty raster</br>
>extract tilename</br>
>Read in gedi data (from shapefile)</br>
>Get list of geometries for all features in GEDI file</br>
>Rasterize vector using the shape and coordinate system of the raster</br>
>save raster using tilename</br>

In [ ]:
empty_raster_rootdir = '/home/jupyter/Sentinel2_Data/GEDI_empty_raster'
gedi_data_rootdir ='/home/jupyter/Sentinel2_Data/Filter_GEDI'
rasterize_gedi_rootdir = '/home/jupyter/Sentinel2_Data/rasterize_GEDI'
gedi_flag_rootdir = '/home/jupyter/Sentinel2_Data/flag_GEDI'

In [ ]:
for root, dirs, files in os.walk(empty_raster_rootdir, topdown=True):
    #print(root)
    for fname in files:
        if fname.endswith('.tif'):
            print(fname)
            empty_img = rasterio.open(os.path.join(root,fname))
            # Extract tilename
            tilename =  fname.split('_')[0]
            
            # Read GEDI data
            gedi_files = os.listdir(gedi_data_rootdir)
            gedi_match = [gedi for gedi in gedi_files if (tilename in gedi)][0] # extract filename
            gedi_data = gpd.GeoDataFrame.from_file(os.path.join(gedi_data_rootdir,gedi_match))
            gedi_data = gedi_data.to_crs(32647)
            
            # Get list of geometries for all features in vector file
            geom_canopy = [(shapes,Canopy_Height) for shapes,Canopy_Height in zip(gedi_data.loc[:,'geometry'],gedi_data.loc[:,'Canopy_height'])]
            geom_flag = [(shapes,flag) for shapes,flag in zip(gedi_data.loc[:,'geometry'],gedi_data.loc[:,'urban_flag'])]
            
            # Rasterize vector using the shape and coordinate system of the raster
            canopy_rasterized = features.rasterize(geom_canopy,
                                        out_shape = empty_img.shape,
                                        fill = 255,
                                        out = None,
                                        transform = empty_img.transform,
                                        all_touched = False,
                                        default_value = 1,
                                        merge_alg = MergeAlg.replace,
                                        dtype = np.float64)
            flag_rasterized = features.rasterize(geom_flag,
                            out_shape = empty_img.shape,
                            fill = 255,
                            out = None,
                            transform = empty_img.transform,
                            all_touched = False,
                            default_value = 1,
                            merge_alg = MergeAlg.replace,
                            dtype = np.float64)
            
            # write raster
            # Canopy Height
            with rasterio.open(os.path.join(rasterize_gedi_rootdir,f'{tilename}_rasterize.tiff'), "w",
                                driver = "GTiff",
                                transform = empty_img.transform,
                                crs = empty_img.crs,
                                dtype = rasterio.float64,
                                count = 1,
                                width = empty_img.width,
                                height = empty_img.height) as dst:
                            dst.write(canopy_rasterized, indexes = 1)
            # Flag
            with rasterio.open(os.path.join(gedi_flag_rootdir,f'{tilename}_flag.tiff'), "w",
                                driver = "GTiff",
                                transform = empty_img.transform,
                                crs = empty_img.crs,
                                dtype = rasterio.float64,
                                count = 1,
                                width = empty_img.width,
                                height = empty_img.height) as dst:
                            dst.write(flag_rasterized, indexes = 1)
            
print('Done')

# Random crop and make dataset
# Flow
count = 0</br>

for eachtile</br>
>load median composite norm GeoTiFF</br>
>extract tilename</br>
>load rasterize GEDI data from tile name</br>
>load flag GEDI data from tile name</br>
>random row start and col start</br>
>create patch boundary</br>
>patch rasterize GEDI</br>
>patch sentinel 2
>count number of GEDI sample in patch</br>
>if sample == 0 or  nan value in sentinel 2 : random row start and col start again</br>

>patch GEDI flag</br>
>count += 1</br>
>if count > 4500 (valiaton set)</br>
>> get index of pixel where value of flag == 1 (urban area)</br>
>> change value of GEDI rasterize to 255 (NAN)</br>
>> random name</br>
>> check name in folder (os.listdir)</br>
>> if name is redundant re random name again</br>
>> save GEDI rasterize</br>
>> convert sentinel 2 to float 32
>> save sentinel 2</br>
>> save GEDI flag</br>

>else (training set)</br>
>> random name</br>
>> check name in folder (os.listdir)</br>
>> if name is redundant re random name again</br>
>> convert all data to float 32
>> save GEDI rasterize</br>
>> save sentinel 2</br>
>> save GEDI flag</br>

In [85]:
sentinel2_rootdir = '/home/jupyter/Sentinel2_Data/Composite_Normalize'
rasterize_gedi_rootdir = '/home/jupyter/Sentinel2_Data/rasterize_GEDI'
gedi_flag_rootdir = '/home/jupyter/Sentinel2_Data/flag_GEDI'
targetdata_sentinel2 = '/home/jupyter/unet_canopyheight_estimation/data/processed/sentinel2'
targetdata_gedi = '/home/jupyter/unet_canopyheight_estimation/data/processed/GEDI_rasterized'
targetdata_flag = '/home/jupyter/unet_canopyheight_estimation/data/processed/GEDI_flag'
num_sample = 5000

In [89]:
for root, dirs, files in os.walk(sentinel2_rootdir, topdown=True):
    #print(root)
    for fname in files:
        if fname.endswith('.tif'):

            # found target raster start building dataset
            count = 0
            print(f'Building dataset on {fname}')
            sentinel2 = rasterio.open(os.path.join(root,fname))
            # Extract tilename
            tilename =  fname.split('_')[0]            
            # load median composite norm GeoTiFF
            sentinel2_img = sentinel2.read()

            # load rasterize GEDI data from tile name
            gedi_files = os.listdir(rasterize_gedi_rootdir)
            gedi_match = [gedi for gedi in gedi_files if (tilename in gedi)][0] # extract filename
            gedi = rasterio.open(os.path.join(rasterize_gedi_rootdir,gedi_match))
            gedi_img = gedi.read()

            #load flag GEDI data from tile name
            gedi_flag_files = os.listdir(gedi_flag_rootdir)
            gedi_flag_match = [gedi for gedi in gedi_flag_files if (tilename in gedi)][0] # extract filename
            gedi_flag = rasterio.open(os.path.join(gedi_flag_rootdir,gedi_flag_match))
            gedi_flag_img = gedi_flag.read()
                
            for i in tqdm(range(num_sample)):
                # random row start and col start
                row_start = random.randint(0, 10980-244)
                col_start = random.randint(0, 10980-244)

                # patch rasterize GEDI
                gedi_subset = gedi_img[:,row_start:row_start+244,col_start:col_start+244].copy()

                # patch sentinel 2
                sentinel2_subset = sentinel2_img[:,row_start:row_start+244,col_start:col_start+244].copy()

                # if all GEDI pixel = 255 (all NAN) or sentine 2 has any pixel = 0(NAN) : random row start and col start again
                while((np.all(gedi_subset==255)) or (np.any(sentinel2_subset==0))):
                    # random row start and col start
                    row_start = random.randint(0, 10980-244)
                    col_start = random.randint(0, 10980-244)
                    # patch rasterize GEDI
                    gedi_subset = gedi_img[:,row_start:row_start+244,col_start:col_start+244].copy()

                    # patch sentinel 2
                    sentinel2_subset = sentinel2_img[:,row_start:row_start+244,col_start:col_start+244].copy()

                # patch GEDI flag
                gedi_flag_subset = gedi_flag_img[:,row_start:row_start+244,col_start:col_start+244].copy()
                
                count += 1 
                
                #if count > 4500 (valiaton set)
                if count > 4500:
                    #get index of pixel where value of flag == 1 (urban area)
                    urban_flag_idx = np.where(gedi_flag_subset == 1)
                    band = urban_flag_idx[0]
                    row  = urban_flag_idx[1]
                    col = urban_flag_idx[2]
                    # change value of GEDI rasterize inurban area to 255 (NAN)
                    gedi_subset[band,row,col] = 255
                    #random name
                    random_name = strgen.StringGenerator("[\w\d]{10}").render()
                    filename = random_name + '.npy'
                    # Check name in folder (os.listdir)
                    # File List
                    name_list_train = os.listdir(os.path.join(targetdata_sentinel2,'train'))
                    name_list_val = os.listdir(os.path.join(targetdata_sentinel2,'validation'))
                    #if name is redundant random name again
                    while((filename in name_list_train) or (filename in name_list_val)):
                        random_name = strgen.StringGenerator("[\w\d]{10}").render()
                        filename = random_name + '.npy'
                    # change all data to float 32
                    sentinel2_subset = sentinel2_subset.astype('float32')
                    gedi_subset = gedi_subset.astype('float32')
                    gedi_flag_subset = gedi_flag_subset.astype('float32')
                    
                    # save all data
                    sentinel2_train_dir = os.path.join(targetdata_sentinel2,'validation')
                    gedi_train_dir = os.path.join(targetdata_gedi,'validation')
                    gedi_flag_train_dir = os.path.join(targetdata_flag,'validation')
                    
                    np.save(os.path.join(sentinel2_train_dir,filename), sentinel2_subset)
                    np.save(os.path.join(gedi_train_dir,filename), gedi_subset)
                    np.save(os.path.join(gedi_flag_train_dir,filename), gedi_flag_subset)
                    
                else: # training set
                    #random name
                    random_name = strgen.StringGenerator("[\w\d]{10}").render()
                    filename = random_name + '.npy'
                    # Check name in folder (os.listdir)
                    # File List
                    name_list_train = os.listdir(os.path.join(targetdata_sentinel2,'train'))
                    name_list_val = os.listdir(os.path.join(targetdata_sentinel2,'validation'))
                    #if name is redundant random name again
                    while((filename in name_list_train) or (filename in name_list_val)):
                        random_name = strgen.StringGenerator("[\w\d]{10}").render()
                        filename = random_name + '.npy'
                        
                    # change all data to float 32
                    sentinel2_subset = sentinel2_subset.astype('float32')
                    gedi_subset = gedi_subset.astype('float32')
                    gedi_flag_subset = gedi_flag_subset.astype('float32')
                    
                    # save all data
                    sentinel2_train_dir = os.path.join(targetdata_sentinel2,'train')
                    gedi_train_dir = os.path.join(targetdata_gedi,'train')
                    gedi_flag_train_dir = os.path.join(targetdata_flag,'train')
                    
                    np.save(os.path.join(sentinel2_train_dir,filename), sentinel2_subset)
                    np.save(os.path.join(gedi_train_dir,filename), gedi_subset)
                    np.save(os.path.join(gedi_flag_train_dir,filename), gedi_flag_subset)

Building dataset on 47PMT_composite_norm.tif


100%|██████████| 5000/5000 [00:39<00:00, 125.29it/s]


Building dataset on 47PQQ_composite_norm.tif


100%|██████████| 5000/5000 [00:52<00:00, 95.45it/s] 


Building dataset on 47PQR_composite_norm.tif


...

100%|██████████| 5000/5000 [01:35<00:00, 52.48it/s]


Building dataset on 47QMA_composite_norm.tif


100%|██████████| 5000/5000 [02:01<00:00, 41.30it/s]


Building dataset on 47QMB_composite_norm.tif


100%|██████████| 5000/5000 [02:09<00:00, 38.64it/s]


In [87]:
gedi_files = os.listdir(rasterize_gedi_rootdir)

In [88]:
gedi_files

['47PQS_rasterize.tiff',
 '47PRR_rasterize.tiff',
 '47PMS_rasterize.tiff',
 '47PQR_rasterize.tiff',
 '47PMT_rasterize.tiff',
 '47QMB_rasterize.tiff',
 '47PQQ_rasterize.tiff',
 '47QMA_rasterize.tiff',
 '47PNS_rasterize.tiff']